In [1]:
from pynq import Overlay
import numpy as np
import random
from pynq import allocate

overlay = Overlay('/home/xilinx/pynq/overlays/aes_axi4/aes.bit')
overlay?

In [2]:
aes_ip = overlay.aes_0
aes_ip?

In [3]:
aes_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  plaintext_1 = Register(plaintext=write-only),
  plaintext_2 = Register(plaintext=write-only),
  ciphertext_1 = Register(ciphertext=write-only),
  ciphertext_2 = Register(ciphertext=write-only),
  key_1 = Register(key=write-only),
  key_2 = Register(key=write-only),
  key_size = Register(key_size=write-only),
  decryptedtext_1 = Register(decryptedtext=write-only),
  decryptedtext_2 = Register(decryptedtext=write-only)
}

In [4]:
plaintext_buffer = allocate(shape=(16,), dtype=np.uint32)
plaintext_array = [ord(char) for char in "abcdef1234567890"]
plaintext_buffer[:] = plaintext_array
key_buffer = allocate(shape=(16,), dtype=np.uint32)
key_array = [ord(char) for char in "kkkkeeeeyyyy...."]
key_buffer[:] = key_array
ciphertext_buffer = allocate(shape=(16,), dtype=np.uint32)
decrypted_plaintext_buffer = allocate(shape=(16,), dtype=np.uint32)

In [5]:
aes_ip.write(0x10, 16) # Write Key Size = 16
aes_ip.register_map.key_1 = key_buffer.physical_address
aes_ip.register_map.plaintext_1 = plaintext_buffer.physical_address
aes_ip.register_map.ciphertext_1 = ciphertext_buffer.physical_address
aes_ip.register_map.decryptedtext_1 = decrypted_plaintext_buffer.physical_address

In [6]:
aes_ip.register_map.CTRL.AP_START=1

In [7]:
print(key_buffer)
print(plaintext_buffer)
print(ciphertext_buffer)
print(decrypted_plaintext_buffer)

[107 107 107 107 101 101 101 101 121 121 121 121  46  46  46  46]
[ 97  98  99 100 101 102  49  50  51  52  53  54  55  56  57  48]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [8]:
aes_ip.register_map.CTRL.AP_START

1

In [9]:
aes_ip.register_map.CTRL

Register(AP_START=1, AP_DONE=0, AP_IDLE=0, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0)

In [10]:
del plaintext_buffer
del key_buffer
del ciphertext_buffer